In [29]:
!pip install torch

     |████████████████████████████████| 147.1 MB 8.9 MB/s 


In [31]:
pip install transformers

     |████████████████████████████████| 3.1 MB 3.5 MB/s 
     |████████████████████████████████| 59 kB 4.4 MB/s 
     |████████████████████████████████| 288 kB 6.1 MB/s 
     |████████████████████████████████| 76 kB 12.0 MB/s 
     |████████████████████████████████| 192 kB 37.7 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 2.2 MB 40.4 MB/s 
     |████████████████████████████████| 306 kB 54.2 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [52]:
pip install --upgrade pip setuptools

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)
     |████████████████████████████████| 946 kB 3.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 52.0.0.post20210125
    Uninstalling setuptools-52.0.0.post20210125:
      Successfully uninstalled setuptools-52.0.0.post20210125
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install ja_sentence_segmenter

Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install fugashi

     |████████████████████████████████| 284 kB 3.9 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install ipadic

     |████████████████████████████████| 13.4 MB 3.8 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=af6411450b4bd0cf00e31c2f968bc70d25051f6830af56fddd6f01ce40e75ad6
  Stored in directory: /Users/yottan/Library/Caches/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully built ipadic
Note: you may need to restart the kernel to use updated packages.


In [56]:
import json
import sys


sys.path.append('..')
import liked_tweets
import pandas as pd
import numpy as np
import sentence_vectorizer
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation



## 特定のキーワードを含むツイートを100件取得

In [50]:
with open('search_tweet.json', 'r', encoding='utf-8') as json_file:
    tweets = json.load(json_file)

# for tweet in tweets['data']:
#     print(tweet['public_metrics']['like_count'])

# df_json = pd.DataFrame(tweets['data'])
# df_json['public_metrics'].values.shape
#['like_count']
# 入れ子の辞書をデータフレームに変換    
df_tweets = pd.json_normalize(tweets['data'])
df_tweets

,author_id,id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
0,311973397,1455913455366512642,ただ、ふと疑問として浮かんだのが、Kagglerって、年金生活になっても Kaggle やる...,0,0,0,0
1,141995203,1455907102912757768,Kaggleのdiscussion，no rankな人の割合が以前より増えている気がする？み...,0,0,0,0
2,1180355783386484736,1455906849190912011,来年からkaggleコンペに一緒に出る仲間を見つけた,0,0,0,0
3,1443902218000683013,1455906592264650755,kaggle初めてみようと思って、Kaggleのチュートリアルを読んでみた。kindle u...,0,0,0,0
4,1351637497,1455903652527230983,kaggleのアドベントカレンダー、すぐ埋まる前にと思って勢いで登録しちゃったけど意外と埋ま...,0,0,1,0
...,...,...,...,...,...,...,...
95,1428519932862492672,1455727129140756482,私もこのアカウントでkaggleやろうかな,0,0,12,0
96,1394978900418076673,1455726920016941056,ご報告いたします。私、運命的な出逢いをして家に迎え入れることにしました。\n\n電子ピアノを...,0,0,28,0
97,1303866497043849217,1455724350758215681,とりあえずしんどいことはわかったわ🐥\n\n#kaggle,0,0,0,0
98,1303866497043849217,1455724089675374598,kaggleやり切ったけど\n\n途中から何やっていいか\nわからんくなった\n\n結局、\...,0,1,0,0


In [51]:
df_tweets = df_tweets.sort_values(['public_metrics.like_count'], ascending=False)
df_tweets[:10]

,author_id,id,text,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
87,1065802066134794240,1455735547767320580,K_matさんの解法凄すぎる。\nhttps://t.co/pzaCp44KVk,6,1,57,0
91,1204025792587386880,1455730254731878408,見れるかな。画像はるの苦労した。英語いい加減でごめんなさい。\n\nhttps://t.co...,4,2,55,0
75,58772914,1455756269633953803,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...,6,1,39,0
74,3246429865,1455756551600304128,自称老害マネージャーがkaggle禁止してるツイート消えちゃったのか。会社のリソースでkag...,4,0,37,0
64,1200396284764381185,1455775679056789506,1st凄すぎた。\n私も頑張りたい。\nhttps://t.co/OrbeTwOTVX,2,0,33,1
52,1457232577,1455810660055486466,Numeraiもそうだけど、kaggleの仮想通貨コンペも生リターンからmarket sig...,2,1,29,0
96,1394978900418076673,1455726920016941056,ご報告いたします。私、運命的な出逢いをして家に迎え入れることにしました。\n\n電子ピアノを...,0,0,28,0
88,234274755,1455735484861149184,@ganntyuebifly すごい…アメリカンドリームだ…！\nスカウトされる理由が気にな...,4,1,22,0
72,1010490727,1455757822151430148,これまでKaggle興味なかった強者が参加した結果だと思うと、普及の先駆けとなったであろうD...,1,0,20,0
29,1165473976463659015,1455837434172739585,日本がKaggle大国という認識進んでいるっぽい？,0,0,18,0


In [52]:
# 関心のあるツイートを選択する
df_tweets.iloc[2]

author_id                                                                58772914
id                                                            1455756269633953803
text                            お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...
public_metrics.retweet_count                                                    6
public_metrics.reply_count                                                      1
public_metrics.like_count                                                      39
public_metrics.quote_count                                                      0
Name: 75, dtype: object

## 特定のツイートにいいねしたユーザー取得

In [53]:
with open('liking_users.json', 'r', encoding='utf-8') as json_file:
    users = json.load(json_file)

# 入れ子の辞書をデータフレームに変換    
df_users = pd.json_normalize(users['data'])
df_users


,name,id,created_at,username,description,location,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count
0,coffee,92283445,2009-11-24T14:24:46.000Z,coffee_pote,元コマンドー 研究メモブログ:http://t.co/WQyWd5ewBX,(:3[____]),784,870,152745,40
1,ばたけ,1127530524224831488,2019-05-12T11:06:40.000Z,batake0708,研究者 兼 データサイエンティスト／ロボット、データ分析/#kaggle Expert(ソロ...,"Tsukuba-shi, Ibaraki",203,174,28,1
2,Takayuki Suzuki,265902560,2011-03-14T09:44:11.000Z,suzuki_takaykey,どうも、神奈川工科大学の鈴木孝幸です。,神奈川県厚木市,270,841,1965,5
3,えるにゃん@ピアノ🎹オケ書く人🎻📯,953231050452254720,2018-01-16T11:42:47.000Z,elnyan4869,『魅せるピアノ』シリーズ/曲を創ります/夢はプロの劇伴作家。毎日ピアノネタ発信してます。/北...,NaN,90,168,320,0
4,クサーブル・クエスト,450821535,2011-12-30T18:06:50.000Z,QsavrQuest,絵が上手くなりたいです。よろしくおねがいします。\nPIXIV: https://t.co/...,NaN,265,3028,318,3
5,Ei-chan,1229321268505133056,2020-02-17T08:26:55.000Z,Ei_chan2,2010年 製造業勤務 I 2017年 アメリカ出向 I 2020年 帰任 I 2021年...,Japan,19,106,770,3
6,ななせまるにあわせまる！！❣️保乃るん麗奈有美子推し,2572557025,2014-06-17T08:52:15.000Z,awasemaru,基本RTばっかする垢です^_^,"Osaka,Japan",2171,4994,799532,31
7,utantar,1073926755394572289,2018-12-15T13:04:25.000Z,utantar,ダフメイト🍚🐿\nゲームしたりたまにアニメの感想とか\n最近は主にアズレン\n雰囲気で生きて...,NaN,594,4853,4790,7
8,トミー＞8 Weeks after Fully Vax,297938726,2011-05-13T11:36:49.000Z,ytommy405,元某出版社の校閲部員。ポケモンGOはTL45（一旦中止中）、赤チームでコードは3517943...,東京、文京区、千代田区、新宿区,241,264,18571,7
9,id5n,1095966567324622848,2019-02-14T08:42:46.000Z,id5n1,Freelance #DataAnalyst || #WebDeveloper: New o...,"New York, NY",318,918,2078,2


In [54]:
user_series = df_users['id']
user_series

0                92283445
1     1127530524224831488
2               265902560
3      953231050452254720
4               450821535
5     1229321268505133056
6              2572557025
7     1073926755394572289
8               297938726
9     1095966567324622848
10             1409197658
11             1017518462
12               41763070
13    1202735068373905409
14    1394639823994953728
15              155513313
16     892732157294030848
17                1765001
18     997661687124639746
19               14971545
20             1406759929
21     808704571488907264
22    1224688117019377664
23    1038406176927444993
24              145333510
25              139674238
26    1113219084307402752
27     973152660671971328
28    1314903407455420422
29    1066948367731777537
30              268813592
31               63173248
32              122638890
33               60845356
Name: id, dtype: object

In [57]:
# API叩く
# liked_tweets.get_liked_tweets(user_series)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
{
    "data": [
        {
            "author_id": "1161797133424455680",
            "id": "1456099775544115203",
            "lang": "ja",
            "text": "\u865a\u69cb\u3067\u91ce\u515a\u306a\u3069\u3092\u653b\u6483\u3057\u3066\u3044\u305f\u30a2\u30ab\u30a6\u30f3\u30c8\u300cDappi\u300d\u306e\u554f\u984c\u3002\u30b9\u30af\u30fc\u30d7\u3092\u51fa\u3057\u3066\u3044\u308b\u306e\u304c\u5831\u9053\u304c\u672c\u696d\u3067\u3042\u308b\u7b48\u306e\u30de\u30b9\u30e1\u30c7\u30a3\u30a2\u3067\u306f\u306a\u304f\u653f\u515a\u6a5f\u95a2\u7d19\u3067\u3042\u308b\u8d64\u65d7\u3068\u3044\u3046\u306e\u304c\u3001\u672c\u90a6\u30de\u30b9\u30e1\u30c7\u30a3\u30a2\u306e\u51cb\u843d\u3076\u308a\u3092\u5982\u5b9f\u306b\u7269\u8a9e\u3063\u3066\u3044\u308b\u3002\u5148\u65e5\u30aa\u30fc\u30b9\u30c8\u30ea\u30a2\u9996\u76f8\u304c\u8f9e\u4efb\u306b\u8ffd\u3044\u8fbc\u307e\u308c\

In [58]:
with open('liked_tweets.json', 'r', encoding='utf-8') as json_file:
    dic_liked_tweets = json.load(json_file)

df_liked_tweets = pd.concat({k: pd.DataFrame(v['data']) for k, v in dic_liked_tweets.items()},axis=0).unstack(0).swaplevel(1,0, axis=1).sort_index(axis=1)
df_liked_tweets


1017518462                            \
             author_id                   id lang   
0   874063400074067970  1455936297516765187   ja   
1   874063400074067970  1455935011756994563   ja   
2   874063400074067970  1456228839990837251   ja   
3   874063400074067970  1456231512714534913   ja   
4   874063400074067970  1456227999200677898   ja   
..                 ...                  ...  ...   
95           121012608  1455773845617446919   ja   
96  727299133430595584  1456042586133516290   ja   
97           126497698  1456042941022044161   ja   
98  737656150838022148  1455867624173170691   ja   
99                 NaN                  NaN  NaN   

                                                       1038406176927444993  \
                                                 text            author_id   
0   という話をしてましたね。世界中から、すさまじいのが集まってる世界でしょうし、多少IQが高い程...   736127313981542401   
1   ワイフさんがMIT（マサチューセッツ工科大）でパーフェクトGPAにベストレポートとoutst...  1006127261307711488   
2   あの時期、足の裏と掌が全面ひび割れて何箇所も血が滲むひどさでしたが\n\n先生から「2以上が...  1428220169927548932   
3   親に\n「あの教師の授業\n　出る気はない、\n　どうおもう？」と\n親に話したら\n\n親...   736127313981542401   
4   小6で陸上全国大会に出る程度の身体能力はありましたが、中1で希少疾患の掌蹠角化症（今は完治）...   729352044465651712   
..                                                ...                  ...   
95  しかしこういう行為には脳を使う必要が無いので、正義を行使していると錯覚して高揚感を覚えたい人...  1006127261307711488   
96  同い歳の者ですがそんな上手な切り返し出来るの素敵だなって憧れる https://t.co/i...  1123209511215349760   
97  ぼくの米株も絶好調でこれだけ株価上がってたらみんな儲かってて家でも車でも買おうかって景気よく...  1006127261307711488   
98  @shoukax @nhsokz イメージとしては、災害現場の体育館に乗り込んでジェンダー平...  1006127261307711488   
99                                                NaN            243798036   

                              \
                     id lang   
0   1456240936313245707   ja   
1   1456240469701136388  und   
2   1456240273202188290   ja   
3   1456240130620985344   ja   
4   1456239830971551747   ja   
..                  ...  ...   
95  1455828414997282817   ja   
96  1455827858853556227   ja   
97  1455820509984133127   ja   
98  1455827138381172737   ja   
99  1455755528857001990   ja   

                                                       1066948367731777537  \
                                                 text            author_id   
0   @LikedReply @k2Bo5rRSaaJaDk5 @TEAR_ACCEL @m_k_...           2953966938   
1   @aurXCanJyw3ePxS @TEAR_ACCEL @ticktack08241 @L...  1440710918434799621   
2   @TEAR_ACCEL @pmari50959 @Ys_Category @KikaiRen...            206180663   
3   @LikedReply @aurXCanJyw3ePxS @uTMCnYabOZcRgWa ...   946093766300987392   
4   @pmari50959 @Ys_Category @KikaiRen @k2Bo5rRSaa...  1240599524390793216   
..                                                ...                  ...   
95  @yoko43445416 @TEAR_ACCEL @LikedReply @uTMCnYa...  1094420053909327873   
96  @Magmag40887776 @TEAR_ACCEL @LikedReply @uTMCn...            113556467   
97  @k2Bo5rRSaaJaDk5 @KikaiRen @ticktack08241 @aur...            113556467   
98  @LikedReply @uTMCnYabOZcRgWa @r6hDIvj5MycWMYt ...  1033729267949953024   
99  なお、初期のD614G変異も、塩基配列上は少し離れているが、蛋白質で折りたたまれたときにフー...             18001121   

                         ... 953231050452254720  \
                     id  ...               lang   
0   1456162716733743110  ...                 ja   
1   1455753921289277447  ...                 ja   
2   1456083447970242561  ...                 ja   
3   1455728552112967682  ...                 ja   
4   1455807458966835205  ...                 ja   
..                  ...  ...                ...   
95  1451063430299877377  ...                 ja   
96  1451065439283343363  ...                und   
97  1451065433637806087  ...                NaN   
98  1451052410466570240  ...                NaN   
99  1451047799768944642  ...                NaN   

                                                        973152660671971328  \
                                                 text            author_id   
0   ショパンが最後に弾いたピアノ🎹\n前回同様にこのピアノの前に立つと、とっても不思議な感覚にな...  1277248231215

In [60]:
# いいねしたツイート文を抽出
# ユーザー名をカラム名にセット
df_liked_tweets_text = pd.concat([df_liked_tweets[k]['text'].rename(k) for k, _ in dic_liked_tweets.items()],axis=1)
df_liked_tweets_text

,92283445,1127530524224831488,265902560,953231050452254720,450821535,1229321268505133056,2572557025,1073926755394572289,297938726,1095966567324622848,...,145333510,139674238,1113219084307402752,973152660671971328,1314903407455420422,1066948367731777537,268813592,63173248,122638890,60845356
0,OMG!!! The RED Aurora moments ago were incredi...,圧力コンペ，(上位解法はともかく)RobustScalerが強かったり，foldたくさん作っ...,今日の東京は14人 https://t.co/VPwdsXFTC9 https://t.co...,ショパンが最後に弾いたピアノ🎹\n前回同様にこのピアノの前に立つと、とっても不思議な感覚にな...,【R-18/NSFW】種付けプレス【閲覧注意】 https://t.co/p6lUQcKdJm,今月から東京都のデジタル化のお手伝いをすることになりました。引き続きよろしくお願いいたします...,「継続できる自信がない」って不安になるくらいなら最初からやらん方がええと思う。 正直、絶対...,嘘を嘘と見抜ける人でないと（ドラえもんを名乗るのは）難しい,日本製紙、印刷用紙など15％以上値上げ 22年出荷分から https://t.co/jNgN...,"Ok, the Anne PRO II is a great little keyboard...",...,Colaboratoryにはいろいろと面白い使い方が見つかる。これはGoogleの関連サイト...,CFP: Design Computing and Cognition 2022\n htt...,月刊アーマーモデリングの机特集みたいに、software design誌かweb+db誌でリ...,おい、朝日新聞。読売や産経と似たような紙面ならわざわざカネ出して読む価値ないんだよ。長い目で...,墨の彩色中。\n\nただ下手過ぎ。\n凄く苦手なんです。\n\nはみ出した箇所は消せるけど本...,当然のことですよね。ワクチン接種済みで隔離措置免除を 経団連が提言案まとめる | NHKニュ...,#今年も残りわずかなので今年一番伸びた絵を貼る見た絵描きさんも強制でやる\nエミリアでした！...,ねじの武勇伝 https://t.co/TXDy4xv2Ve,@ochitohfu 埋葬させてもらえないなど、酷い罰を受けましたね…,虚構で野党などを攻撃していたアカウント「Dappi」の問題。スクープを出しているのが報道が本...
1,東京維新の全体会議。衆院選を戦った仲間が集まりました。皆さん、それぞれの立場で懸命に戦い抜い...,atmaも圧力コンペもTransformer...\nTransformer実装できるように...,維新政治の本質―その支持層についての考察\nhttps://t.co/74VkfaH88x\...,紅葉の中で白鳥が羽ばたいた奇跡の瞬間なので見て欲しい。 https://t.co/BEdSn...,https://t.co/WakyEyJYVC\n過去に描いた絵pixiv版,弊娘のチアリーディングの練習にきてるのだけど、同じく練習が終わるのを待ってる目の前のお母さん...,Autumn day https://t.co/15rOTCCto0,TIMEより\nコロナ発生後ほぼ二年を経てその発生過程を解明するのが難しくなっていることを受...,ご迷惑と思いながら思わずリツイート。実は私は「免除職」最後の世代で、「免除職」がなくなった世...,christian girl autumn tech bro on top // techn...,...,高崎ハンパない。\nそんなんできひんやん普通。\n\n高崎市がヤングケアラーにヘルパー無料派...,東大・情報理工・20周年記念シンポジウム開催のご案内\n https://t.co/C95l...,一つの企業内ですらこうなんだから、企業と企業で比べたらもっと大きな差があるの、当たり前なんだよな,佐川急便でまたパワハラ自殺、上司が激しい叱責「お前どれだけ嘘つくんだよ」 会社は謝罪 htt...,リトアニアのIPA、ライトだけど美味い。世界のビールが手頃な値段で手に入るのは嬉しい。勿論H...,どうやって暴走族の名前が떡볶이w https://t.co/o8VC8vyNni,Twitterくんほんと何考えてるか分からない… https://t.co/2rc7Hi8z8w,【社主のニュース解説】《社会》\n胸を刺された原首相は、わずかに目を開き、息も絶え絶えに。好...,小説などに影響されて事件を起こすという話題になるとよくゲーテの『若きウェルテルの悩み』が取り...,労弁の水口先生の分析。データに基づく鋭い分析です。\n\n2021年総選挙に思う 衆院比例区...
2,これが博士号取得者しか乗船を許されないという伝説のボート......！ https://t....,kaggle 圧力コンペ終わったので、今日から競プロリハビリ開始！\nと思ったけどkaggl...,研究室の学部生がFacebookから年棒1500万円+株+入社ボーナス600万円のオファーも...,メイちゃんのポケットから無限に出てくるオモチャ🦀🦑🐟 #鳥羽水族館 #ラッコ https:/...,エクスポート機能がすんなりQuestに移植出来て嬉しい,GoogleBrain圧力コンペ、初めてのチーム戦で18位で銀メダル獲得です！\n参加者のみ...,太田光代さんの何が凄いって、乗っ取られたアカウントを取り返せたことだよね\n乗っ取り犯は乗っ...,「求められた以上の仕事をして驚かせるぞ」というプロ意識をもっているが、同時に「貰う金額以上の...,【スタッフよる（園芸係）】\n園内では、紅（黄）葉が始まっています。メタセコイアやラクウショ...,Method A is 10x faster and more complex than M...,...,本のための日本郵便占い作りました🔮 https://t.co/mZ6JPQYSBB,早速来たので勉強します https://t.co/IBfojqXVJB,［定期宣伝］\nSoftware Design ＆ WEB+DB PRESS年間購読ボーナス...,「非自民で右に需要あるぞ」\n「おぅ！」\nで、そこに傾注してみたところで。\n\nメディア...,黒田長興の来訪について書かれた翠巌の手紙の執筆年を調べるため、寛永15年～寛文3年(1638...,おうち帰ってベッドに沈んでダメになりながら韓国ドラマを見るのサイコォ〜だな？？,ミニョネット号事件を今日も扱ったわけだが、重すぎるテーマのため、努めて冗談を混ぜたり、明るい...,コーヒー初心者でも選べるコーヒー産地のフローチャートつくってみました☕️ https://t...,曽根崎心中がヒットして心中が流行ったので幕府が心中物禁止令を出してる,歯周病 江戸時代もかかっていた！ 現代とは異なる細菌が原因 | NHKニュース http...
3,@godfather_part1 ぜひとも面接に落ちて伝説になってほしいですw,圧力コンペ900位台だったのでもうデータサイエンス辞めようかと思ったけど、解法眺めてたら謎に...,【円谷英二展 応援コメント！】 \n桜井浩子さん（俳優／『ウルトラQ』江戸川由利子役、『ウル...,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...,PlayAniMaker for Quest can now export animatio...,motoさんでも「1日に何件も面接をしていた」という部分が良かった。「面接に疲弊→妥協してキ...,ホッとしたと同時に\n素敵な一日になりました,洗濯機にポケットティッシュが袋ごと混入していたらしく、満面の笑みになっている。これしきのこと...,先日子供が見ていたテレビ(Eテレ？)から「神様の言うとおり」と聞こえてきましたが、「言うとお...,Caring about others within a company in a capi...,...,「東京の生活史」を寝る前に読み進めているのですが、芸能人とかそういうんじゃなく誰でもない誰か...,Ph.D.取った方が給料もペイするし、民間企業もPh.D.もち採用して伸びてるらしいぜ！って...,先月のSoftwareDesignのデータモデリング特集読んでたんだけど、理論学ばず、手癖で...,定年が75歳になれば年金制度は崩壊しないって、それを崩壊というんじゃねえか馬鹿野郎！！,翠巌宗珉は、千宗旦の肖像画に賛を書いた人物でもある（絵は松花堂昭乗 筆）。翠巌は宗旦より30...,最近は中国のSNSで色んな情報を仕入れてるのだけど、外国人として生きてくのはカナダvsアメリ...,ぼくに感情的に反発する人はこんな感じの人が多いですね。 https://t.co/KjLwg...,進捗がなさすぎて虚無虚無プリン,出会いが再会みたいなことが立て続けに起きて、11

In [ ]:
sample_df = pd.DataFrame(['お腹が痛いので遅れます。',
                    '頭が痛いので遅れます。',
                    'おはようございます！',
                    'kaggle が好きなかえる',
                    '味噌汁が好きなワニ'
                              ], columns=['text'])

In [61]:
# ユーザー一人のいいねしたツイート
df_liked_tweets_text['1017518462']

0     という話をしてましたね。世界中から、すさまじいのが集まってる世界でしょうし、多少IQが高い程...
1     ワイフさんがMIT（マサチューセッツ工科大）でパーフェクトGPAにベストレポートとoutst...
2     あの時期、足の裏と掌が全面ひび割れて何箇所も血が滲むひどさでしたが\n\n先生から「2以上が...
3     親に\n「あの教師の授業\n　出る気はない、\n　どうおもう？」と\n親に話したら\n\n親...
4     小6で陸上全国大会に出る程度の身体能力はありましたが、中1で希少疾患の掌蹠角化症（今は完治）...
                            ...                        
95    しかしこういう行為には脳を使う必要が無いので、正義を行使していると錯覚して高揚感を覚えたい人...
96    同い歳の者ですがそんな上手な切り返し出来るの素敵だなって憧れる https://t.co/i...
97    ぼくの米株も絶好調でこれだけ株価上がってたらみんな儲かってて家でも車でも買おうかって景気よく...
98    @shoukax @nhsokz イメージとしては、災害現場の体育館に乗り込んでジェンダー平...
99                                                  NaN
Name: 1017518462, Length: 100, dtype: object

## いいねしたツイート文の前処理

In [80]:
# 元のツイート文の前処理

target_texts = df_tweets.iloc[2]['text']
target_texts

'お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficientNetV2でベースラインモデルを作ってからチューニングまではできそうだが、5章のGradiaentTapeクラスが新規の知識が必要で勉強がいる。'

In [81]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

target_texts = pd.DataFrame(segmenter(target_texts), columns=['text'])
target_texts

,text
0,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。
1,素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficie...


In [192]:
BSV = sentence_vectorizer.BertSequenceVectorizer()
target_texts['vector'] = target_texts['text'].progress_map(lambda x: BSV.vectorize(x))
target_texts['vector']

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


0    [-0.57061076, -0.1908873, -0.19887878, -0.6743...
1    [-0.2974286, 0.31751335, -0.462103, -0.3291492...
Name: vector, dtype: object

In [18]:
target_texts

,text,vector
0,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。,"[-0.57061076, -0.1908873, -0.19887878, -0.6743..."
1,素材がKMNISTでくずし字なので、なんとかなるかと思って読んでいるが、4章のEfficie...,"[-0.2974286, 0.31751335, -0.462103, -0.3291492..."


In [115]:
# ユーザーごとに文を結合するので、ツイート文末に'。'をつける
df_liked_tweets_text += '。'
df_liked_tweets_text

,92283445,1127530524224831488,265902560,953231050452254720,450821535,1229321268505133056,2572557025,1073926755394572289,297938726,1095966567324622848,...,145333510,139674238,1113219084307402752,973152660671971328,1314903407455420422,1066948367731777537,268813592,63173248,122638890,60845356
0,OMG!!! The RED Aurora moments ago were incredi...,圧力コンペ，(上位解法はともかく)RobustScalerが強かったり，foldたくさん作っ...,今日の東京は14人 https://t.co/VPwdsXFTC9 https://t.co...,ショパンが最後に弾いたピアノ🎹\n前回同様にこのピアノの前に立つと、とっても不思議な感覚にな...,【R-18/NSFW】種付けプレス【閲覧注意】 https://t.co/p6lUQcKdJm。,今月から東京都のデジタル化のお手伝いをすることになりました。引き続きよろしくお願いいたします...,「継続できる自信がない」って不安になるくらいなら最初からやらん方がええと思う。 正直、絶対...,嘘を嘘と見抜ける人でないと（ドラえもんを名乗るのは）難しい。,日本製紙、印刷用紙など15％以上値上げ 22年出荷分から https://t.co/jNgN...,"Ok, the Anne PRO II is a great little keyboard...",...,Colaboratoryにはいろいろと面白い使い方が見つかる。これはGoogleの関連サイト...,CFP: Design Computing and Cognition 2022\n htt...,月刊アーマーモデリングの机特集みたいに、software design誌かweb+db誌でリ...,おい、朝日新聞。読売や産経と似たような紙面ならわざわざカネ出して読む価値ないんだよ。長い目で...,墨の彩色中。\n\nただ下手過ぎ。\n凄く苦手なんです。\n\nはみ出した箇所は消せるけど本...,当然のことですよね。ワクチン接種済みで隔離措置免除を 経団連が提言案まとめる | NHKニュ...,#今年も残りわずかなので今年一番伸びた絵を貼る見た絵描きさんも強制でやる\nエミリアでした！...,ねじの武勇伝 https://t.co/TXDy4xv2Ve。,@ochitohfu 埋葬させてもらえないなど、酷い罰を受けましたね…。,虚構で野党などを攻撃していたアカウント「Dappi」の問題。スクープを出しているのが報道が本...
1,東京維新の全体会議。衆院選を戦った仲間が集まりました。皆さん、それぞれの立場で懸命に戦い抜い...,atmaも圧力コンペもTransformer...\nTransformer実装できるように...,維新政治の本質―その支持層についての考察\nhttps://t.co/74VkfaH88x\...,紅葉の中で白鳥が羽ばたいた奇跡の瞬間なので見て欲しい。 https://t.co/BEdSn...,https://t.co/WakyEyJYVC\n過去に描いた絵pixiv版。,弊娘のチアリーディングの練習にきてるのだけど、同じく練習が終わるのを待ってる目の前のお母さん...,Autumn day https://t.co/15rOTCCto0。,TIMEより\nコロナ発生後ほぼ二年を経てその発生過程を解明するのが難しくなっていることを受...,ご迷惑と思いながら思わずリツイート。実は私は「免除職」最後の世代で、「免除職」がなくなった世...,christian girl autumn tech bro on top // techn...,...,高崎ハンパない。\nそんなんできひんやん普通。\n\n高崎市がヤングケアラーにヘルパー無料派...,東大・情報理工・20周年記念シンポジウム開催のご案内\n https://t.co/C95l...,一つの企業内ですらこうなんだから、企業と企業で比べたらもっと大きな差があるの、当たり前なんだよな。,佐川急便でまたパワハラ自殺、上司が激しい叱責「お前どれだけ嘘つくんだよ」 会社は謝罪 htt...,リトアニアのIPA、ライトだけど美味い。世界のビールが手頃な値段で手に入るのは嬉しい。勿論H...,どうやって暴走族の名前が떡볶이w https://t.co/o8VC8vyNni。,Twitterくんほんと何考えてるか分からない… https://t.co/2rc7Hi8z8w。,【社主のニュース解説】《社会》\n胸を刺された原首相は、わずかに目を開き、息も絶え絶えに。好...,小説などに影響されて事件を起こすという話題になるとよくゲーテの『若きウェルテルの悩み』が取り...,労弁の水口先生の分析。データに基づく鋭い分析です。\n\n2021年総選挙に思う 衆院比例区...
2,これが博士号取得者しか乗船を許されないという伝説のボート......！ https://t....,kaggle 圧力コンペ終わったので、今日から競プロリハビリ開始！\nと思ったけどkaggl...,研究室の学部生がFacebookから年棒1500万円+株+入社ボーナス600万円のオファーも...,メイちゃんのポケットから無限に出てくるオモチャ🦀🦑🐟 #鳥羽水族館 #ラッコ https:/...,エクスポート機能がすんなりQuestに移植出来て嬉しい。,GoogleBrain圧力コンペ、初めてのチーム戦で18位で銀メダル獲得です！\n参加者のみ...,太田光代さんの何が凄いって、乗っ取られたアカウントを取り返せたことだよね\n乗っ取り犯は乗っ...,「求められた以上の仕事をして驚かせるぞ」というプロ意識をもっているが、同時に「貰う金額以上の...,【スタッフよる（園芸係）】\n園内では、紅（黄）葉が始まっています。メタセコイアやラクウショ...,Method A is 10x faster and more complex than M...,...,本のための日本郵便占い作りました🔮 https://t.co/mZ6JPQYSBB。,早速来たので勉強します https://t.co/IBfojqXVJB。,［定期宣伝］\nSoftware Design ＆ WEB+DB PRESS年間購読ボーナス...,「非自民で右に需要あるぞ」\n「おぅ！」\nで、そこに傾注してみたところで。\n\nメディア...,黒田長興の来訪について書かれた翠巌の手紙の執筆年を調べるため、寛永15年～寛文3年(1638...,おうち帰ってベッドに沈んでダメになりながら韓国ドラマを見るのサイコォ〜だな？？。,ミニョネット号事件を今日も扱ったわけだが、重すぎるテーマのため、努めて冗談を混ぜたり、明るい...,コーヒー初心者でも選べるコーヒー産地のフローチャートつくってみました☕️ https://t...,曽根崎心中がヒットして心中が流行ったので幕府が心中物禁止令を出してる。,歯周病 江戸時代もかかっていた！ 現代とは異なる細菌が原因 | NHKニュース http...
3,@godfather_part1 ぜひとも面接に落ちて伝説になってほしいですw。,圧力コンペ900位台だったのでもうデータサイエンス辞めようかと思ったけど、解法眺めてたら謎に...,【円谷英二展 応援コメント！】 \n桜井浩子さん（俳優／『ウルトラQ』江戸川由利子役、『ウル...,お休みなので、ソフトウェアデザイン誌の「Kaggleで知る機械学習」で勉強。素材がKMNIS...,PlayAniMaker for Quest can now export animatio...,motoさんでも「1日に何件も面接をしていた」という部分が良かった。「面接に疲弊→妥協してキ...,ホッとしたと同時に\n素敵な一日になりました。,洗濯機にポケットティッシュが袋ごと混入していたらしく、満面の笑みになっている。これしきのこと...,先日子供が見ていたテレビ(Eテレ？)から「神様の言うとおり」と聞こえてきましたが、「言うとお...,Caring about others within a company in a capi...,...,「東京の生活史」を寝る前に読み進めているのですが、芸能人とかそういうんじゃなく誰でもない誰か...,Ph.D.取った方が給料もペイするし、民間企業もPh.D.もち採用して伸びてるらしいぜ！って...,先月のSoftwareDesignのデータモデリング特集読んでたんだけど、理論学ばず、手癖で...,定年が75歳になれば年金制度は崩壊しないって、それを崩壊というんじゃねえか馬鹿野郎！！。,翠巌宗珉は、千宗旦の肖像画に賛を書いた人物でもある（絵は松花堂昭乗 筆）。翠巌は宗旦より30...,最近は中国のSNSで色んな情報を仕入れてるのだけど、外国人として生きてくのはカナダvsアメリ...,ぼくに感情的に反発する人はこんな感じの人が多いですね。 https://t.co/KjLwg...,進捗がなさすぎて虚無虚無プリン。,出会いが再会

In [119]:
#転置してユーザーIDをインデックスにする
df_liked_tweets_text_t = df_liked_tweets_text.T

df_liked_tweets_text_t['all'] = df_liked_tweets_text_t.iloc[:,0].str.cat([df_liked_tweets_text_t.iloc[:,col] 
for col in df_liked_tweets_text_t.columns[1:]],na_rep='')

In [193]:
# 結合されているか確認
df_liked_tweets_text_t['all']

92283445               OMG!!! The RED Aurora moments ago were incredi...
1127530524224831488    圧力コンペ，(上位解法はともかく)RobustScalerが強かったり，foldたくさん作っ...
265902560              今日の東京は14人 https://t.co/VPwdsXFTC9 https://t.co...
953231050452254720     ショパンが最後に弾いたピアノ🎹\n前回同様にこのピアノの前に立つと、とっても不思議な感覚にな...
450821535              【R-18/NSFW】種付けプレス【閲覧注意】 https://t.co/p6lUQcKdJ...
1229321268505133056    今月から東京都のデジタル化のお手伝いをすることになりました。引き続きよろしくお願いいたします...
2572557025             「継続できる自信がない」って不安になるくらいなら最初からやらん方がええと思う。  正直、絶対...
1073926755394572289    嘘を嘘と見抜ける人でないと（ドラえもんを名乗るのは）難しい。嘘を嘘と見抜ける人でないと（ドラ...
297938726              日本製紙、印刷用紙など15％以上値上げ　22年出荷分から https://t.co/jNgN...
1095966567324622848    Ok, the Anne PRO II is a great little keyboard...
1409197658             どこの国にどうやって行こう？\n国決めとビザの話🇳🇿📃（3） https://t.co/8i...
1017518462             という話をしてましたね。世界中から、すさまじいのが集まってる世界でしょうし、多少IQが高い程...
41763070               今日はリビングに8人もいる。。今日はリビングに8人もいる。。@IY0YI やっと求めてるポイ...
1202735068373905409    ベイビーわるきゅーれ売り切れてます！！！！！！！！！！！

In [178]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_no = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(の)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_no, split_punc2)

df_list = []
for idx, text in enumerate(df_liked_tweets_text_t['all']):
    df_list.append(list(segmenter(text)))

df_liked_tweets_text_t['segments'] = df_list    

[['OMG!',
  '!',
  '!',
  'The RED Aurora moments ago were incredible near Alix Alberta!',
  '!',
  '#Aurora#TeamTanner@treetanner@TamithaSkov@chunder10@NightLights_AM@weathernetwork@scottrockphoto@JimCantore@MatthewCappucci@mikesobel@PeakToSailPhoto@RedDeerAdvocate@CTVJoelle@Sunny94FM@KyleTWN https://t.co/8I2spFUqB3。',
  'OMG!',
  '!',
  '!',
  'The RED Aurora moments ago were incredible near Alix Alberta!',
  '!',
  '#Aurora#TeamTanner@treetanner@TamithaSkov@chunder10@NightLights_AM@weathernetwork@scottrockphoto@JimCantore@MatthewCappucci@mikesobel@PeakToSailPhoto@RedDeerAdvocate@CTVJoelle@Sunny94FM@KyleTWN https://t.co/8I2spFUqB3。',
  '東京維新の全体会議。',
  '衆院選を戦った仲間が集まりました。',
  '皆さん、それぞれの立場で懸命に戦い抜いてくれた。',
  '本当に有難う。',
  '今後に向けた決意も。',
  '所属国会議員は5名となり重責を担うことに。',
  '役割を果たします。',
  'https://t.co/GYBnCoiqoM。',
  'これが博士号取得者しか乗船を許されないという伝説のボート......!',
  'https://t.co/DPPNC1K6le。',
  '@godfather_part1ぜひとも面接に落ちて伝説になってほしいですw。',
  '@suzumetamashiいろんなトラブルあるでしょうねえwお店側は大変ですね。',
  '@dama_ojこんな奴らまで採用せざる

In [ ]:
df_liked_tweets_text_t['segments'][0]

In [190]:
BSV = sentence_vectorizer.BertSequenceVectorizer()

vectors_list = []
vectors_list.append([pd.Series(text).progress_map(lambda x: BSV.vectorize(x)) for text in df_liked_tweets_text_t['segments'][:2]])


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 309/309 [03:27<00:00,  1.49it/s]


In [191]:
vectors_list

[[0      [-0.4512673, -0.04766576, 0.2516311, -0.557453...
  1      [-0.18984501, -0.093700156, -0.34990737, -0.33...
  2      [-0.18984501, -0.093700156, -0.34990737, -0.33...
  3      [-0.38991755, -0.58017904, 0.030932426, -0.478...
  4      [-0.18984501, -0.093700156, -0.34990737, -0.33...
                               ...                        
  247    [-0.18984501, -0.093700156, -0.34990737, -0.33...
  248    [-0.20613123, 0.08262817, -0.12085131, -0.3086...
  249    [-0.2741638, 0.2928401, -0.46076638, -0.076426...
  250    [-0.7732281, -0.15554862, -0.7126255, -0.34860...
  251    [-0.45997214, 0.37671435, -0.4111026, -0.04918...
  Length: 252, dtype: object,
  0      [-0.33005786, 0.28634775, -0.02408741, -0.3187...
  1      [-0.33005786, 0.28634775, -0.02408741, -0.3187...
  2      [-0.35065717, 0.22759047, -0.07796751, -0.2964...
  3      [-0.13760094, 0.13294177, 0.046310663, -0.4022...
  4      [0.09970313, -0.10282055, -0.017816897, -0.188...
                          